In [3]:
from collections import defaultdict
from fractions import Fraction
from math import factorial
from operator import itemgetter


def binomial(n, k):
    return factorial(n) // (factorial(k) * factorial(n - k))


def find_path(graph, s, t):
    stack = [s]
    predecessor = {s: t}
    while stack:
        v = stack.pop()
        for u in graph[v]:
            if u not in predecessor:
                stack.append(u)
                predecessor[u] = v
    assert t in predecessor
    path = [t]
    while path[-1] != s:
        path.append(predecessor[path[-1]])
    path.reverse()
    return path


def round_flow(flow):
    while True:
        capacities = []
        for (u, v), x in flow.items():
            z = x - x.numerator // x.denominator
            if z:
                capacities.append(((v, u), z))
                capacities.append(((u, v), 1 - z))
        if not capacities:
            break
        (t, s), delta = min(capacities, key=itemgetter(1))
        graph = defaultdict(list)
        for (v, u), z in capacities:
            if (v, u) not in [(s, t), (t, s)]:
                graph[v].append(u)
        path = find_path(graph, s, t)
        for i, v in enumerate(path):
            u = path[i - 1]
            if (u, v) in flow:
                flow[(u, v)] += delta
            else:
                flow[(v, u)] -= delta


def baranyai(n, k):
    m, r = divmod(n, k)
    assert not r, 'n (%s) must be divisible by k (%s)' % (n, k)

    M = binomial(n - 1, k - 1)
    partition = [[()] * m for i in range(M)]
    for l in range(n):
        flow = defaultdict(Fraction)
        for i, A_i in enumerate(partition):
            for S in A_i:
                flow[(i, S)] += Fraction(k - len(S), n - l)
        round_flow(flow)
        next_partition = []
        for i, A_i in enumerate(partition):
            next_A_i = []
            for S in A_i:
                if flow[(i, S)]:
                    next_A_i.append(S + (l,))
                    flow[(i, S)] -= 1
                else:
                    next_A_i.append(S)
            next_partition.append(next_A_i)
        partition = next_partition
    assert len(partition) == M
    classes = set()
    for A in partition:
        assert len(A) == m
        assert all(len(S) == k for S in A)
        assert len({x for S in A for x in S}) == n
        classes.update(map(frozenset, A))
    assert len(classes) == binomial(n, k)
    return partition

In [31]:
rows = [
    [(0, 1, 2, 3), (4, 5, 6, 7)],
    [(0, 2, 4, 6), (1, 3, 5, 7)],
    [(0, 1, 3, 6), (2, 4, 5, 7)],

    
    [(0, 5, 6, 7), (1, 2, 3, 4)],
    
    
    [(0, 1, 3, 4), (2, 5, 6, 7)],
    [(0, 2, 4, 7), (1, 3, 5, 6)],
    [(0, 2, 4, 5), (1, 3, 6, 7)],
    [(0, 2, 3, 6), (1, 4, 5, 7)],
    [(0, 2, 3, 7), (1, 4, 5, 6)],
    [(0, 1, 3, 7), (2, 4, 5, 6)],
    [(0, 3, 4, 5), (1, 2, 6, 7)],
    [(0, 4, 5, 6), (1, 2, 3, 7)],
    [(0, 2, 3, 4), (1, 5, 6, 7)],
    [(0, 1, 5, 6), (2, 3, 4, 7)],
    [(0, 3, 5, 7), (1, 2, 4, 6)],
    [(0, 2, 5, 7), (1, 3, 4, 6)],
    [(0, 4, 6, 7), (1, 2, 3, 5)],
    [(0, 2, 3, 5), (1, 4, 6, 7)],
    [(0, 3, 4, 6), (1, 2, 5, 7)],
    [(0, 1, 4, 5), (2, 3, 6, 7)],
    [(0, 3, 5, 6), (1, 2, 4, 7)],
    [(0, 1, 4, 6), (2, 3, 5, 7)],
    [(0, 1, 5, 7), (2, 3, 4, 6)],
    [(0, 1, 2, 4), (3, 5, 6, 7)],
    [(0, 2, 6, 7), (1, 3, 4, 5)],
    [(0, 2, 5, 6), (1, 3, 4, 7)],
    [(0, 2, 4, 6), (1, 3, 5, 7)],
    [(0, 1, 2, 7), (3, 4, 5, 6)],
    [(0, 1, 2, 5), (3, 4, 6, 7)],
    [(0, 1, 4, 7), (2, 3, 5, 6)],
    [(0, 1, 3, 5), (2, 4, 6, 7)],
    [(0, 3, 4, 7), (1, 2, 5, 6)],
    [(0, 1, 2, 6), (3, 4, 5, 7)],
    [(0, 4, 5, 7), (1, 2, 3, 6)],
    [(0, 1, 6, 7), (2, 3, 4, 5)],
]

In [50]:
print(get_good(taken_edges))

None


In [48]:
def get_good(taken_edges):
    taken_set = set()
    for edge in taken_edges:
        for group in edge:
            i, j, k, l = group
            taken_set.add((i, j))
            taken_set.add((j, k))
            taken_set.add((k, l))

    for row in rows:
        if row_is_okay(taken_set):
            return row
    return None
                
def row_is_okay(taken):
    for group in row:
        i, j, k, l = group
        if (i, j) in taken:
            return False
        if (j, k) in taken:
            return False
        if (k, l) in taken:
            return False
    return True

In [13]:
rows = r"""$a^{\dagger}_7 a^{\dagger}_5 a_3 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_4 a_2 a_1$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_5 a_3 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_4 a_2 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_6 a_3 a_0$ \quad $a^{\dagger}_5 a^{\dagger}_4 a_2 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_4 a_3 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_5 a_2 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_5 a_4 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_3 a_2 a_1$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_4 a_3 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_2 a_1$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_5 a_4 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_3 a_2 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_6 a_4 a_0$ \quad $a^{\dagger}_5 a^{\dagger}_3 a_2 a_1$ \\ \hline 
$a^{\dagger}_5 a^{\dagger}_4 a_3 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_2 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_6 a_5 a_0$ \quad $a^{\dagger}_4 a^{\dagger}_3 a_2 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_5 a_1 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_4 a_3 a_2$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_5 a_2 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_4 a_3 a_1$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_5 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_4 a_3 a_2$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_5 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_4 a_3 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_6 a_1 a_0$ \quad $a^{\dagger}_5 a^{\dagger}_4 a_3 a_2$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_4 a_1 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_5 a_3 a_2$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_6 a_2 a_0$ \quad $a^{\dagger}_5 a^{\dagger}_4 a_3 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_3 a_1 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_5 a_4 a_2$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_4 a_2 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_5 a_3 a_1$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_4 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_3 a_2$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_3 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_4 a_2$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_3 a_2 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_5 a_4 a_1$ \\ \hline 
$a^{\dagger}_5 a^{\dagger}_3 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_4 a_2$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_4 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_3 a_1$ \\ \hline 
$a^{\dagger}_5 a^{\dagger}_4 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_3 a_2$ \\ \hline 
$a^{\dagger}_4 a^{\dagger}_3 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_5 a_2$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_3 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_4 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_2 a_1 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_5 a_4 a_3$ \\ \hline 
$a^{\dagger}_5 a^{\dagger}_3 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_4 a_1$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_2 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_4 a_3$ \\ \hline 
$a^{\dagger}_5 a^{\dagger}_2 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_4 a_3$ \\ \hline 
$a^{\dagger}_5 a^{\dagger}_4 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_3 a_1$ \\ \hline 
$a^{\dagger}_4 a^{\dagger}_2 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_5 a_3$ \\ \hline 
$a^{\dagger}_4 a^{\dagger}_3 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_5 a_1$ \\ \hline 
$a^{\dagger}_3 a^{\dagger}_2 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_5 a_4$ \\ \hline 
""".split('hline')

In [14]:
import random
random.shuffle(rows)

In [18]:
for row in rows:
    print(row + 'hline', end="")

 
$a^{\dagger}_6 a^{\dagger}_4 a_3 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_2 a_1$ \\ \hline 
$a^{\dagger}_7 a^{\dagger}_5 a_2 a_0$ \quad $a^{\dagger}_6 a^{\dagger}_4 a_3 a_1$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_5 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_4 a_3 a_1$ \\ \hline 
$a^{\dagger}_3 a^{\dagger}_2 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_5 a_4$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_5 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_4 a_3 a_2$ \\ \hline 
$a^{\dagger}_5 a^{\dagger}_3 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_4 a_2$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_4 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_3 a_1$ \\ \hline 
$a^{\dagger}_5 a^{\dagger}_3 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_6 a_4 a_1$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_4 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_3 a_2$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_2 a_1 a_0$ \quad $a^{\dagger}_7 a^{\dagger}_5 a_4 a_3$ \\ \hline 
$a^{\dagger}_6 a^{\dagger}_3 a_2 a_0$ \quad $a^{\dagger}_7 a^{\dagge